```console
$ pip install pyvista
$ pip install trame
$ pip install trame-vtk
$ pip install trame-vuetify
$ pip install ipywidgets
```

In [1]:
import pyvista
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

pyvista.set_jupyter_backend('client')

In [2]:
def read_colormap_file(filename, name="custom", reverse=False):
    colormap_data = np.loadtxt(filename)
    if reverse:
        colormap_data = colormap_data[::-1]
    cmap = LinearSegmentedColormap.from_list(name, colormap_data)
    return cmap

In [3]:
data = pyvista.read("./vis/reg_1_alpha_kernel.vtk")

In [4]:
# boundaries = pyvista.read("/home/orsvuran/Documents/AVS_continent_boundaries.vtu")

In [5]:
def spherical_to_cartesian(lat, lon, r):
    theta = np.pi / 2 - np.deg2rad(lat)
    phi = np.deg2rad(lon)
    return [
        r * np.sin(theta) * np.cos(phi),
        r * np.sin(theta) * np.sin(phi),
        r * np.cos(theta),
    ]


def get_normalized_depth(depth, R=6371.0):
    """Turns depth to Radius of the unit sphere."""
    return (R - depth) / R


def get_normal_cartesian(x0, y0, z0, x1, y1, z1):
    a = np.array([x0, y0, z0])
    b = np.array([x1, y1, z1])
    n = np.cross(a, b)
    n = n / np.linalg.norm(n)
    return n


def get_plane_normal_cartesian(x0, y0, z0, x1, y1, z1):
    a = np.array([x0, y0, z0])
    b = np.array([x1, y1, z1])
    n = np.cross(a, b)
    n = n / np.linalg.norm(n)
    return n


def get_plane_normal_spherical(lat0, lon0, lat1, lon1, d0=0, d1=0, R=6371.0):
    loc0 = spherical_to_cartesian(lat0, lon0, get_normalized_depth(d0, R))
    loc1 = spherical_to_cartesian(lat1, lon1, get_normalized_depth(d1, R))
    return get_plane_normal_cartesian(*loc0, *loc1)

In [6]:
cmap = read_colormap_file("./vis/red_yellow_cyan_blue.dat", reverse=True)

In [7]:
normal = get_plane_normal_spherical(37.49, 137.17, 1.3549, 172.9229, d0=12.0)

In [8]:
slice1 = data.slice(normal, origin=(0, 0, 0))

In [9]:
p = pyvista.Plotter()
p.add_mesh(slice1, clim=(-1e-6, 1e-6), cmap=cmap)
source = pyvista.Sphere(0.02, center=spherical_to_cartesian(37.49, 137.17, get_normalized_depth(12.0)))
station = pyvista.Sphere(0.02, center=spherical_to_cartesian(1.3549, 172.9229, get_normalized_depth(0.0)))
p.add_mesh(source)
p.add_mesh(station)
# p.add_mesh(boundaries, color="black")
p.camera_position = [(-0.6006338601038901, -1.0514154287074078, 1.5359628467148005),
 (-0.35779793560504913, 0.3092064131051302, 0.40716348122805357),
 (-0.669778474894765, 0.5412746567173775, 0.5083488374759283)]
p.show()

Widget(value='<iframe src="http://localhost:40661/index.html?ui=P_0x7ed21b5822a0_0&reconnect=auto" class="pyvi…